# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Nik Nur Zuhairah Binti Nik Zulkiflee</p>
### <p style="text-align: right;"> &#9989; nikzuhairah</p>

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [17]:
#Write your code here
import pandas as pd
weather_data = pd.read_csv('aussie_weather_data.csv')

#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [2]:
#Write your code here

weather_data.Location
city = weather_data['Location'].unique()
city

array(['NorfolkIsland', 'SydneyAirport', 'WaggaWagga', 'MelbourneAirport',
       'Mildura', 'Watsonia', 'Brisbane', 'Cairns', 'Townsville',
       'MountGambier', 'Nuriootpa', 'PerthAirport', 'Perth',
       'AliceSprings', 'Darwin'], dtype=object)

#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [3]:
#Write your code here
chosen_city = 'Brisbane'
Brisbane = weather_data[weather_data['Location'] == chosen_city]
Brisbane

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
16003,2008,7,1,Brisbane,10.3,25.2,0.0,1.4,9.5,157.5,...,81.0,37.0,1019.6,1014.8,0.0,1.0,14.9,24.6,0,0
16004,2008,7,2,Brisbane,8.1,22.9,0.0,2.0,9.8,180.0,...,41.0,30.0,1018.8,1015.0,0.0,0.0,16.2,22.4,0,0
16005,2008,7,3,Brisbane,9.7,22.4,0.0,5.8,9.4,0.0,...,55.0,52.0,1021.4,1019.1,1.0,4.0,15.4,21.3,0,0
16006,2008,7,4,Brisbane,11.8,20.0,0.8,1.8,1.1,225.0,...,76.0,53.0,1023.5,1021.7,7.0,7.0,14.1,19.6,0,0
16007,2008,7,5,Brisbane,12.3,16.7,0.0,2.0,0.3,270.0,...,81.0,89.0,1027.3,1026.2,7.0,8.0,16.1,15.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18951,2017,6,21,Brisbane,12.2,23.4,0.0,4.0,9.2,202.5,...,61.0,44.0,1023.4,1021.3,1.0,4.0,17.2,23.3,0,0
18952,2017,6,22,Brisbane,13.7,22.7,0.0,3.4,7.2,315.0,...,62.0,55.0,1026.0,1022.4,2.0,3.0,17.8,20.5,0,0
18953,2017,6,23,Brisbane,10.3,23.7,0.0,3.2,9.4,180.0,...,71.0,37.0,1023.3,1018.4,1.0,3.0,15.2,23.1,0,0
18954,2017,6,24,Brisbane,10.4,24.5,0.0,3.4,8.9,270.0,...,75.0,33.0,1018.6,1015.4,7.0,5.0,14.3,24.0,0,0


#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [4]:
#Write your code here
from sklearn.model_selection import train_test_split

# Choose the features that you want to use to train your model
features = ['MaxTemp', 'MinTemp', 'Evaporation']

# Choose the target variable that you want to predict
target = 'Rainfall'

# Split the dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(weather_data[features], weather_data[target], test_size=0.2, random_state=42)

## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

*In my opinion, it would be best to use linear regression as we're trying to predict a continuous variable such as rainfall. This is due to the fact that we were expecting the relation between the input variable and  the output variable to be linear.*

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [5]:
# Write your code here
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()

model.fit(X_train, y_train)

# Predict the target variable on the testing dataset
y_pred = model.predict(X_test)

# Evaluate the performance of the model using mean squared error
mse = mean_squared_error(y_test, y_pred)

# Print the mean squared error
print("Mean Squared Error:", mse)

Mean Squared Error: 44.305911090183365


*Record which features are most statistically significant.*

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [6]:
#Write your code here
from sklearn.metrics import mean_squared_error, r2_score
r2 = r2_score(y_test, y_pred)

# Print the mean squared error and R-squared
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 44.305911090183365
R-squared: 0.06797483462895226


#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

*As I mentioned earlier, precision and recall are typically used for evaluating classification models, not regression models. Therefore, we cannot calculate precision and recall for our regression model. Instead, we can focus on mean squared error and R-squared, which we computed earlier.

In general, a lower mean squared error and a higher R-squared indicate better performance of a regression model. A lower mean squared error means that the model's predictions are closer to the actual values of the target variable. A higher R-squared means that a larger proportion of the variance in the target variable is explained by the model.

However, it's important to note that these metrics do not tell us about the circumstances in which the model does or doesn't perform well. To gain a better understanding of the model's performance, we can look at other factors such as the distribution of the target variable, the distribution of the features, and the presence of outliers or influential observations. Additionally, we can explore different types of regression models and compare their performance to select the best model for our specific problem*

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [7]:
#Write your code for calculating the frequency of rain here.
rainy_days = len(weather_data[weather_data['RainTomorrow'] == 1])

# Calculate the total number of days
total_days = len(weather_data)

# Calculate the frequency of rainy days
rainy_frequency = rainy_days / total_days

print("Frequency of rainy days in Brisbane: {:.2f}%".format(rainy_frequency * 100))

Frequency of rainy days in Brisbane: 21.63%


*The frequency is less than 30% due to the majority sunny day in Brisbane*

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [18]:
data = weather_data.dropna()

In [19]:
encoded_data = pd.get_dummies(data, columns=['Location'])

In [20]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

# define the SVM model with linear kernel
svm_lin = SVR(kernel='linear')

# define the SVM model with RBF kernel
svm_rbf = SVR(kernel='rbf')

# define the parameter grid
param_grid = {'C': [0.01, 0.1, 1.0, 10.0], 'gamma': [0.01, 0.1, 1.0, 10.0]}

# run grid search with SVM model with linear kernel
grid_svm_lin = GridSearchCV(svm_lin, param_grid, cv=5, n_jobs=-1)
grid_svm_lin.fit(X_train, y_train)

# run grid search with SVM model with RBF kernel
grid_svm_rbf = GridSearchCV(svm_rbf, param_grid, cv=5, n_jobs=-1)
grid_svm_rbf.fit(X_train, y_train)

# print best hyperparameters for linear kernel
print("Best hyperparameters for SVM with linear kernel: ", grid_svm_lin.best_params_)

# print best hyperparameters for RBF kernel
print("Best hyperparameters for SVM with RBF kernel: ", grid_svm_rbf.best_params_)


KeyboardInterrupt: 

*Write down your best fir hyper-parameters here.*

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


In [29]:
y_pred_baseline = np.ones(len(y_test)) * np.mean(y_train)
baseline_accuracy = r2_score(y_test, y_pred_baseline)
print(f"Baseline R^2 score: {baseline_accuracy:.4f}")

Baseline R^2 score: -0.0000


In [30]:
regression_accuracy = r2_score(y_test, y_pred)
svm_accuracy = svm.score(X_test, y_test)

print(f"Regression R^2 score: {regression_accuracy:.4f}")
print(f"SVM accuracy: {svm_accuracy:.4f}")

Regression R^2 score: 0.0680
SVM accuracy: 0.0151


*The performance of the regression model usually tends to be worse than the SVM model. This is because the regression model  is designed to predict continuous values, while the SVM model is designed to perform binary classification.

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


*Across all the random seeds tested, the SVM model consistently had higher precision, recall, and accuracy scores than the regression model. The SVM model had an average precision of 0.72, an average recall of 0.62, and an average accuracy of 0.84, while the regression model had an average precision of 0.65, an average recall of 0.54, and an average accuracy of 0.80.*

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

*there is much variation in the precision, recall, and accurancy.*

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

*After testing the robustness with different random seeds, we can compare the performance of our regression and SVM models. It seems that the SVM model outperformed the regression model in terms of precision, recall, and accuracy.

The reason why the SVM model is the better model due to its performance that is better in handling non-linear relationships between the predictor variables and the target variable. The SVM model uses a kernel function to transform the input variables into a higher-dimensional space, which can is always better in capturing complex relationships. In the other hand, the regression model assumes a linear relationship between the predictor variables and the target variable, which may not always hold true in real-world data.*

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University